In [29]:
from hyptorch.geoopt import PoincareBall   
from hyptorch.lorentz.manifold import CustomLorentz 
from model.modules.utils import ManifoldMapper
import torch


poincare = PoincareBall(c=1.0, learnable=True)
lorentz = CustomLorentz(k=1.0, learnable=True)

lmapper = ManifoldMapper(lorentz, clip_r=1.25, curv=lorentz.k)
pmapper = ManifoldMapper(lorentz, clip_r=1.25, curv=poincare.c)
vis_embed = torch.rand(100, 12,256)
text_embed = torch.rand(100,  256)


vis_embed = lmapper(vis_embed)
text_embed = lmapper(text_embed)
lorentz.assert_check_point_on_manifold(vis_embed)
# text_embed = text_embed.unsqueeze(1).unsqueeze(1)
text_embed = text_embed.unsqueeze(-1) 
print('text_embed', text_embed.shape)
# vis_embed = vis_embed.permute(0, 2, 1)
vis_embed = vis_embed.unsqueeze(1) 
print('vision_embed', vis_embed.shape)

# dist = lorentz.dist_batch(text_embed, vis_embed)
simi2q = lorentz.dist_batch(vis_embed, text_embed)





text_embed torch.Size([100, 257, 1])
vision_embed torch.Size([100, 1, 12, 257])


In [ ]:
import torch.nn.functional as F
def itc_loss(self, image_embeds , text_embeds):

    sim_q2t = torch.matmul(
        image_embeds.unsqueeze(1), text_embeds.unsqueeze(-1)
    ).squeeze()
    # [batch_size, batch_size*num_gpu, num_query_tokens]

    # image-text similarity: aggregate across all query tokens
    sim_i2t, _ = sim_q2t.max(-1)
    sim_i2t = sim_i2t / self.temp

    # text-query similarity: [batch_size, batch_size, num_query_tokens]
    sim_t2q = torch.matmul(
        text_embeds.unsqueeze(1).unsqueeze(1), image_embeds.permute(0, 2, 1)
    ).squeeze()

    # text-image similarity: aggregate across all query tokens
    sim_t2i, _ = sim_t2q.max(-1)
    sim_t2i = sim_t2i / self.temp  # [batch_size, batch_size*num_gpu]

    bs = image_embeds.size(0)
    targets = torch.arange(bs).to(self.device)

    itc_loss= (
        F.cross_entropy(sim_i2t, targets, label_smoothing=0.1)
        + F.cross_entropy(sim_t2i, targets, label_smoothing=0.1)
    ) / 2
    stats = {
        "logits/weight_t2i": 1.0 - self.weight_i2t,
        "logits/itc_loss": itc_loss.item(),
        "logits/min": sim_i2t.min().item(),
        "logits/mean": sim_i2t.mean().item(),
        "logits/max": sim_i2t.max().item(),
        "logits/acc": (sim_i2t.argmax(-1) == targets).float().mean().item(),
        "logits/curvature": self.manifold.k.item(),
    }
    return itc_loss, stats, sim_i2t

